In [1]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn import tree
from collections import Counter
import math

In [2]:
iris=load_iris()

In [3]:
def entropy(dataset):
    dataset=np.array(dataset)
    labelcount=Counter(dataset[:,-1])
    num=len(dataset)
    ent=-sum([labelcount[i]/num*math.log(labelcount[i]/float(num),2) for i in labelcount])
    return ent
def cond_ent(dataset,axis=0):
    #axis:特征编号，用以计算相应特征下数据集的条件熵
    featuredict={}
    num=len(dataset)
    for i in range(num):
        feature=dataset[i][axis]
        if feature not in featuredict:
            featuredict[feature]=[]
        featuredict[feature].append(dataset[i])
    ent=sum([len(p)/num*entropy(p) for p in featuredict.values()])
    return ent
def info_gain_train(dataset):
    featurenum=len(dataset[0])-1
    maxind,maxent=-1,-1e3
    base_ent=entropy(dataset)
    for i in range(featurenum):
        ent_gain=-cond_ent(dataset,axis=i)+base_ent
        print(ent_gain)
        if ent_gain>maxent:
            maxind=i
            maxent=ent_gain
    return maxind

In [4]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels
dataset,features=create_data()

In [37]:
class node(object):
    def __init__(self,feature=None,root=False,label=None,featurename=None):
        self.root=root         #判断叶子节点
        self.label=label       #标签     **
        self.feature=feature   #选择特征 **
        self.featurename=featurename
        self.tree={}           #存储特征选择后的所有子节点的字典
        self.result = {        
            '特征选择': self.featurename,
            'label:': self.label,
            '子节点': self.tree
            
        }

    def add_node(self,feature,node):
        self.tree[feature]=node
        
    def __repr__(self):
        return '{}'.format(self.result)
    def predict(self,features):
        # 若为叶子节点则返回标签
        if self.root:
            return self.label
        #返回本节点特征选择后子节点的预测
        if self.feature:        
            #子集删除特征列后，预测序列应同子集保持一致
            featurenew=features[:self.feature]
            featurenew.extend(features[self.feature+1:])
            return self.tree[features[self.feature]].predict(featurenew)
        else:
            return self.tree[features[self.feature]].predict(features)
class dtree(object):
    def __init__(self,features,epsion=0.1):
        self.epsion=epsion
        self._tree={}
        self.featurelist=features
        self.road=[]
    def entropy(self,dataset):
        m=len(dataset)
        dataset=np.array(dataset)
        labelcount={}
        for label in dataset[:,-1]:
            if label not in labelcount:
                labelcount[label]=0
            labelcount[label]+=1
        ent=-sum([p/float(m)*math.log(p/float(m),2) for p in labelcount.values()])
        return ent
    def choose_feature(self,dataset):
        base_ent=self.entropy(dataset)        
        m,n=len(dataset),len(dataset[0])-1
        maxent,maxind=0,-1
        for ind in range(n):            
            featuredict={}
            for j in range(m):
                feature=dataset[j][ind]
                if feature not in featuredict:
                    featuredict[feature]=[]
                featuredict[feature].append(dataset[j])
            ent_gain=base_ent-sum([len(p)/float(m)*self.entropy(p) for p in featuredict.values()])
            if ent_gain>maxent:
                maxent=ent_gain
                maxind=ind
        return maxind,maxent
    def train(self,dataset):
        dataset=np.array(dataset)
        labels=dataset[:,-1].tolist()
        #1 若标签仅有一种则直接返回标签
        if labels.count(labels[0])==1:
            return node(root=True,label=labels[0])
        if len(dataset[0])==1:
            label=max(labels,key=labels.count)
            return node(root=True,label=label)       
        bestfeature,maxent=self.choose_feature(dataset)
        if maxent<self.epsion:
            label=max(labels,key=labels.count)
            return node(root=True,label=label)  
        self.road.append(bestfeature)
        tree=node(root=False,feature=bestfeature,featurename=self.featurelist[bestfeature])
        subset={}
        for ind in range(dataset.shape[0]):
            feature=dataset[ind,bestfeature]
            if feature not in subset:
                subset[feature]=[]
            newinstance=dataset[ind,:bestfeature].tolist()
            newinstance.extend(dataset[ind,bestfeature+1:])
            subset[feature].append(newinstance)
        for feature in subset:
            subtrain=np.array(subset[feature])
            subnode=self.train(subtrain)
            tree.add_node(feature=feature,node=subnode)       
        return tree
    def fit(self,dataset):
        self._tree=self.train(dataset)
        return self._tree
    def predict(self,features):
        return self._tree.predict(features)

In [38]:
tree=dtree([u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别'])
dt=tree.fit(dataset)
#print(dt)
print(dt.predict(['老年', '否', '否', '一般']))

否


In [8]:
print(dt.predict(['老年', '否', '否', '一般']))

是


In [23]:
if None:
    print('none')

In [12]:
dt.predict(['青年', '是', '否', '非常好'])

'是'

In [236]:
print(dt.tree)

{'是': {'子节点': {}, 'label:': '是', 'loc:': 2, '特征选择': None}, '否': {'子节点': {'是': {'子节点': {}, 'label:': '是', 'loc:': 4, '特征选择': None}, '否': {'子节点': {}, 'label:': '否', 'loc:': 5, '特征选择': None}}, 'label:': None, 'loc:': 3, '特征选择': '有工作'}}


In [167]:
# 定义节点类 二叉树
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label:': self.label,
            'feature': self.feature,
            'tree': self.tree
        }

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)


class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * math.log(p / data_length, 2)
                    for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p) / data_length) * self.calc_ent(p)
                        for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        y_train=train_data[:,-1].tolist()

        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if y_train.count(y_train[0]) == 1:
            return Node(root=True, label=y_train[0])
        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(root=True,label=max(y_train,key=y_train.count))

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        bestfeature, max_info_gain = self.info_gain_train(np.array(train_data))
  
        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True,label=max(y_train,key=y_train.count))

        # 5,构建Ag子集
        node_tree = Node(root=False, feature=bestfeature)


        subset={}
        for ind in range(train_data.shape[0]):
            feature=train_data[ind,bestfeature]
            if feature not in subset:
                subset[feature]=[]
            newinstance=train_data[ind,:bestfeature].tolist()
            newinstance.extend(train_data[ind,bestfeature+1:])
            subset[feature].append(newinstance)
        for feature in subset:
            sub_train=np.array(subset[feature])
            # 6, 递归生成树
            sub_tree = self.train(sub_train)
            node_tree.add_node(feature, sub_tree)

        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [182]:
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
DT = DTree()
dataset=np.array(datasets)
Tree = DT.fit(dataset)

In [183]:
Tree

{'feature': 2, 'label:': None, 'tree': {'是': {'feature': None, 'label:': '是', 'tree': {}}, '否': {'feature': 1, 'label:': None, 'tree': {'是': {'feature': None, 'label:': '是', 'tree': {}}, '否': {'feature': None, 'label:': '否', 'tree': {}}}}}}

In [180]:
dt.predict(['老年', '否', '否', '一般'])

'否'

In [181]:
a=['否','是','否','是','是','是']
a.count(a[0])

2

In [157]:
max(a,key=a.count)

'是'